In [5]:
import os
from PyPDF2 import PdfReader

def extract_pdf_content(pdf_path):
    """Extracts text from a single PDF file."""
    try:
        reader = PdfReader(pdf_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text() or ""
        return text
    except Exception as e:
        print(f"Failed to read {pdf_path}: {e}")
        return None

def get_pdfs_in_directory(directory):
    """Creates a dictionary of PDF filenames and their content in a directory."""
    pdf_contents = {}
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            file_path = os.path.join(directory, filename)
            content = extract_pdf_content(file_path)
            if content is not None:
                pdf_contents[filename] = content
    return pdf_contents

# Example usage
directory_path = 'data'
pdf_files_content = get_pdfs_in_directory(directory_path)
print(pdf_files_content)


{'merkblatt_kiel_bauordnungsamt_abbruch_von_anlagen.pdf': 'Abbruch von Anlagen\nBeseitigung von (baulichen) Anlagen\nRechtsgrundlage § 61 Abs. 3 Satz 1 Landesbauordnung für das Land Schleswig-Holstein (LBO)\nVerfahrensfrei ist nach § 61 Abs. 3 Satz 1 LBO die Beseitigung von\n –Anlagen nach § 61 Abs. 1 LBO, d. h. solcher Anlagen, deren Errichtung ebenfalls verfahrensfrei ist (Nr. 1),\n –freistehenden Gebäuden der Gebäudeklasse 1 und Gebäudeklasse 3 (Nr. 2),\n –sonstigen Anlagen, die keine Gebäude sind, mit einer Höhe bis zu 10 m (Nr. 3),\nsoweit es sich nicht um Kulturdenkmale nach dem Denkmalschutzgesetz für das Land Schleswig-Holstein \nhandelt.  \n Kieler Denkmalkataster \nGebäudeklassen\n§ 2 Abs. 3 LBO teilt die Gebäude in fünf Gebäudeklassen ein, die sich in der Kombination zwischen  \nmittlerer Fußbodenhöhe des höchstgelegenen Aufenthaltsraumes und Anzahl sowie Größe von Nutzungseinheiten unterscheiden:\n –Gebäudeklasse 1 umfasst  \na) freistehende Gebäude mit einer Höhe bis zu 7 

In [6]:
len(pdf_files_content)

9

In [18]:
import json
from datetime import datetime
import uuid

class Conversation:
    def __init__(self, system_prompt=""):
        self.id = str(uuid.uuid4())
        self.messages = [{"role": "system", "content": system_prompt}]
        self.user_messages = []
        self.start_time = datetime.now()

    def add_exchange(self, prompt, user_message, assistant_message):
        self.messages.extend([{"role": "user", "content": prompt}, {"role": "assistant", "content": assistant_message}])
        self.user_messages.append(user_message)

    def get_conversation_data(self):
        return {
            "id": self.id,
            "start_time": self.start_time.isoformat(),
            "messages": self.messages,
            "user_messages": self.user_messages,
            "end_time": datetime.now().isoformat(),
        }

# Function to log a conversation
def log_conversation(conversation, filename="conversation_log.json"):
    try:
        with open(filename, "r") as file:
            data = json.load(file)
    except FileNotFoundError:
        data = []
    
    data.append(conversation.get_conversation_data())

    with open(filename, "w") as file:
        json.dump(data, file, indent=4)

# Example usage
#conversation = Conversation()
#conversation.add_exchange("Hello, how are you?", "Hello, how are you?", "I'm good, thank you!")
#conversation.add_exchange("What's the weather like?", "What's the weather like?", "It's sunny and warm.")
#log_conversation(conversation)


In [19]:
import json
from datetime import datetime
import uuid

class Inquiry:
    def __init__(self):
        self.id = str(uuid.uuid4())
        self.steps = []
        self.start_time = datetime.now()

    def add_step(self, step):
        self.steps.append(step.get_lookup_data())

    def get_inquiry_data(self):
        return {
            "id": self.id,
            "start_time": self.start_time.isoformat(),
            "steps": self.steps,
            "end_time": datetime.now().isoformat(),
        }

class DocumentLookUp:
    def __init__(self):
        self.id = str(uuid.uuid4())
        self.lookups = []
        self.start_time = datetime.now()

    def add_lookup(self, doc_name, conversation):
        self.lookups.append({doc_name: conversation.get_conversation_data()})

    def get_lookup_data(self):
        return {
            "id": self.id,
            "start_time": self.start_time.isoformat(),
            "lookups": self.lookups,
            "end_time": datetime.now().isoformat(),
        }

# Function to log a conversation
def log_inquiry(conversation, filename="inquiry_log.json"):
    try:
        with open(filename, "r") as file:
            data = json.load(file)
    except FileNotFoundError:
        data = []
    
    data.append(inquiry.get_conversation_data())

    with open(filename, "w") as file:
        json.dump(data, file, indent=4)

In [20]:
from dotenv import load_dotenv

load_dotenv()  # This loads the environment variables from .env

True

In [21]:
question = "Ich möchte meine Garage abreißen"

In [22]:
system_prompt_1 = "Du bist ein konzentrierter und ordentlicher Aufgabenbearbeiter"

In [23]:
from openai import OpenAI
client = OpenAI()

In [24]:
import concurrent.futures

def process_pdf(k, v, question, system_prompt_1, client):
    prompt = "Referenzdokument:\n" + v + "\n\n-----------\n\n" + "Bitte extrahiere alle relevanten Informationen aus dem oben genannten Referenzdokument, die zur folgenden Anfrage passen:\n" + question
    messages = [
        {"role": "system", "content": system_prompt_1},
        {"role": "user", "content": prompt}
    ]
    conversation= Conversation(system_prompt = system_prompt_1)
    
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.0
    )
    conversation.add_exchange(prompt, question, completion.choices[0].message.content)
    step1.add_lookup(k, conversation)
    return (k,completion)

responses = []
inquiry = Inquiry()
step1 = DocumentLookUp()
with concurrent.futures.ThreadPoolExecutor() as executor:
    future_to_pdf = {executor.submit(process_pdf, k, v, question, system_prompt_1, client): k for k, v in pdf_files_content.items()}
    for future in concurrent.futures.as_completed(future_to_pdf):
        pdf_key = future_to_pdf[future]
        try:
            response = future.result()
            responses.append(response)
        except Exception as exc:
            print(f'{pdf_key} generated an exception: {exc}')

inquiry.add_step(step1)


In [26]:
data =inquiry.get_inquiry_data()

In [46]:
data['steps'][0]['lookups'][0]['merkblatt_kiel_bauordnungsamt_verfahrensfreie_nebenanlagen.pdf']['messages']

[{'role': 'system',
  'content': 'Du bist ein konzentrierter und ordentlicher Aufgabenbearbeiter'},
 {'role': 'user',
  'content': 'Referenzdokument:\nVerfahrensfreie Nebenanlagen\nRechtsgrundlage § 61 Abs. 1 Landesbauordnung  \nfür das Land Schleswig-Holstein (LBO)\nBauaufsichtlich verfahrensfrei sind nach § 61 Abs. 1 Nr. 1a LBO Abstellräume, Geräteschuppen, \nGartenlauben, wenn folgende Kriterien erfüllt werden: \n –ohne Aufenthaltsräume, Toiletten und Feuerstätten\n –im Innenbereich (§ 34 BauGB) oder im Geltungsbereich eines Bebauungsplanes (§ 30 BauGB)  \nbis max. 30 m³ umbauter Brutto-Rauminhalt\n –Liegt das Baugrundstück im Geltungsbereich eines Bebauungsplanes, ist zu prüfen, ob die  Errichtung einer Nebenanlage den Festsetzungen des Bebauungsplanes entspricht.\n –im Außenbereich (§ 35 BauGB) bis max. 10 m³ umbauter Brutto-Rauminhalt\n –Im Außenbereich ist ein Vorhaben nur zulässig, wenn öffentliche Belange nicht entgegenstehen,  die ausreichende Erschließung gesichert ist und w

In [47]:
for file, completion in responses:
    print(file)
    
    print(completion.choices[0].message.content)
    print("###############")

merkblatt_kiel_bauordnungsamt_verfahrensfreie_nebenanlagen.pdf
Gemäß § 61 Abs. 1 Nr. 1b LBO sind Garagen bauaufsichtlich verfahrensfrei, solange sie eine mittlere Wandhöhe bis zu 3,00 m und eine Brutto-Grundfläche bis zu 30 m² haben. Wenn du deine Garage abreißen möchtest, musst du sicherstellen, dass du keine Genehmigung benötigst, da sie unter die verfahrensfreien Nebenanlagen gemäß der Landesbauordnung Schleswig-Holstein fallen.
###############
merkblatt_kiel_bauordnungsamt_verfahrensfreie_bauvorhaben.pdf
Gemäß § 61 Abs. 1 Nr. 1b der Landesbauordnung für das Land Schleswig-Holstein (LBO) sind notwendige Garagen und Fahrradgaragen, einschließlich überdachter Stellplätze und Abstellplätze für Fahrräder, in Grenznähe oder auf der Grenze mit bis zu 9,00 m Gesamtlänge (inklusive Dachüberstand) und mit einer mittleren Wandhöhe bis zu 3,00 m über der an der Grundstücksgrenze festgelegten Geländeoberfläche sowie einer Brutto-Grundfläche bis zu 30 m² verfahrensfrei. 

Wenn Sie Ihre Garage ab

In [48]:
info_str = ""
for k, completion in responses:
    info_str+= "\n\n---------\n\n" + completion.choices[0].message.content

In [49]:
print(info_str)



---------

Gemäß § 61 Abs. 1 Nr. 1b LBO sind Garagen bauaufsichtlich verfahrensfrei, solange sie eine mittlere Wandhöhe bis zu 3,00 m und eine Brutto-Grundfläche bis zu 30 m² haben. Wenn du deine Garage abreißen möchtest, musst du sicherstellen, dass du keine Genehmigung benötigst, da sie unter die verfahrensfreien Nebenanlagen gemäß der Landesbauordnung Schleswig-Holstein fallen.

---------

Gemäß § 61 Abs. 1 Nr. 1b der Landesbauordnung für das Land Schleswig-Holstein (LBO) sind notwendige Garagen und Fahrradgaragen, einschließlich überdachter Stellplätze und Abstellplätze für Fahrräder, in Grenznähe oder auf der Grenze mit bis zu 9,00 m Gesamtlänge (inklusive Dachüberstand) und mit einer mittleren Wandhöhe bis zu 3,00 m über der an der Grundstücksgrenze festgelegten Geländeoberfläche sowie einer Brutto-Grundfläche bis zu 30 m² verfahrensfrei. 

Wenn Sie Ihre Garage abreißen möchten, müssen Sie sicherstellen, dass Sie die bauordnungs- und planungsrechtlichen Vorschriften einhalten. 

In [51]:
system_prompt_2 = "Du bist ein hilfreicher Fragenbeantworter einer Baubehörde, der Fragen basierend auf dem zur Verfügung gestellten Kontext aus behördlicher Sicht beantwortet"

In [52]:
from IPython.display import Markdown
prompt ="Information:\n" + info_str + "\n\n-----------\n\n" + "Nutze die bereitgestellten Informationen, um mit den relevanten Informationen die Anfrage nutzerfreundlich zu beantworten. Die Anfrage ist: \n" +question
messages=[
        {"role": "system", "content": system_prompt_2},
      ]
messages+= [{"role": "user", "content": prompt}]
conversation= Conversation(system_prompt = system_prompt_2)
completion = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=messages,
    temperature=0.0
)
conversation.add_exchange(prompt, question, completion.choices[0].message.content)
step2= conversation
inquiry.add_step(step2)

display(Markdown(completion.choices[0].message.content))

AttributeError: 'Conversation' object has no attribute 'get_lookup_data'

In [54]:
from IPython.display import Markdown
step2 = DocumentLookUp()
step2.add_lookup('summary', conversation)
inquiry.add_step(step2)

display(Markdown(completion.choices[0].message.content))

Um Ihre Garage in Schleswig-Holstein abzureißen, sollten Sie zunächst prüfen, ob Ihre Garage die Kriterien für eine verfahrensfreie Beseitigung gemäß § 61 Abs. 1 Nr. 1b und § 61 Abs. 3 Satz 1 der Landesbauordnung Schleswig-Holstein (LBO) erfüllt. Wenn Ihre Garage eine mittlere Wandhöhe von bis zu 3,00 m, eine Brutto-Grundfläche von bis zu 30 m² hat und die Gesamtlänge inklusive Dachüberstand 9,00 m nicht überschreitet, ist der Abriss in der Regel verfahrensfrei.

Trotz der Verfahrensfreiheit empfehle ich Ihnen, sich vor dem Abriss mit der zuständigen Bauaufsichtsbehörde in Verbindung zu setzen, um sicherzustellen, dass keine weiteren bauordnungs- oder planungsrechtlichen Vorschriften betroffen sind. Dies ist besonders wichtig, wenn die Garage direkt an andere Gebäude angrenzt oder spezielle Umstände vorliegen, die eine Genehmigung erforderlich machen könnten.

Für den Abriss benötigen Sie möglicherweise folgende Unterlagen, falls die Garage nicht den verfahrensfreien Kriterien entspricht:
- Einen Lageplan, der die Lage der Garage auf Ihrem Grundstück zeigt.
- Eine Erklärung zur bautechnischen Prüfung, falls erforderlich.
- Eine Anzeige der Beseitigung bei der Bauaufsichtsbehörde, falls die Garage nicht verfahrensfrei abgerissen werden kann.

Ich empfehle Ihnen, sich an das Amt für Bauordnung, Vermessung und Geoinformation in Kiel zu wenden, um eine Beratung zu erhalten und sicherzustellen, dass alle notwendigen Schritte korrekt durchgeführt werden. Sie können das Amt unter der Telefonnummer 0431 901-2660 erreichen oder eine E-Mail an bauaufsicht@kiel.de senden, um weitere Informationen zu erhalten oder einen Beratungstermin zu vereinbaren.

Bitte beachten Sie, dass auch wenn der Abriss verfahrensfrei ist, Sie dennoch verpflichtet sind, die Sicherheit während des Abrisses zu gewährleisten und alle Abfälle und Materialien ordnungsgemäß zu entsorgen.

In [55]:
# Function to log a conversation
def log_inquiry(conversation, filename="inquiry_log.json"):
    try:
        with open(filename, "r") as file:
            data = json.load(file)
    except FileNotFoundError:
        data = []
    
    data.append(inquiry.get_inquiry_data())

    with open(filename, "w") as file:
        json.dump(data, file, indent=4)

In [56]:
log_inquiry(inquiry)

In [53]:
messages

[{'role': 'system',
  'content': 'Du bist ein hilfreicher Fragenbeantworter einer Baubehörde, der Fragen basierend auf dem zur Verfügung gestellten Kontext aus behördlicher Sicht beantwortet'},
 {'role': 'user',
  'content': 'Information:\n\n\n-----------\n\nNutze die bereitgestellten Informationen, um mit den relevanten Informationen die Anfrage nutzerfreundlich zu beantworten. Die Anfrage ist: \nIch möchte meine Garage abreißen'}]

In [ ]:
info_str = ""
for completion in responses:
    info_str+= "\n\n---------\n\n" + completion.choices[0].message.content

In [21]:
print(info_str)



---------

Um Ihre Garage in Schleswig-Holstein abzureißen, müssen Sie folgende relevante Informationen aus dem Referenzdokument beachten:

1. **Verfahrensfreiheit**: Der Abbruch von Anlagen kann verfahrensfrei sein, wenn:
   - Die Garage als freistehendes Gebäude der Gebäudeklasse 1 oder 3 zählt. Gebäudeklasse 1 umfasst freistehende Gebäude mit einer Höhe bis zu 7 m und nicht mehr als zwei Nutzungseinheiten von insgesamt nicht mehr als 400 m². Gebäudeklasse 3 umfasst sonstige Gebäude mit einer Höhe bis zu 7 m.

2. **Denkmalschutz**: Stellen Sie sicher, dass die Garage nicht als Kulturdenkmal nach dem Denkmalschutzgesetz für das Land Schleswig-Holstein eingestuft ist. Informationen dazu können im Kieler Denkmalkataster eingesehen werden.

3. **Anzeige der Beseitigung**: Falls die Garage nicht unter die verfahrensfreien Anlagen fällt, muss die Beseitigung einen Monat zuvor der Bauaufsichtsbehörde angezeigt werden. Nutzen Sie hierfür den Vordruck unter VordruckeLandesbauordnung_Anlage1

In [23]:
system_prompt_2 = "Du bist ein hilfreicher Fragenbeantworter, der Fragen basierend auf dem zur Verfügung gestellten Kontext beantwortet"

In [24]:
prompt ="Information:\n" + info_str + "\n\n-----------\n\n" + "Nutze die bereitgestellten Informationen, um mit den relevanten Informationen die Anfrage nutzerfreundlich zu beantworten. Die Anfrage ist: \n" +question
messages=[
        {"role": "system", "content": system_prompt_2},
      ]
messages+= [{"role": "user", "content": prompt}]
completion = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=messages,
    temperature=0.0
)

print(completion.choices[0].message.content)

Um den Abriss Ihrer Garage in Schleswig-Holstein rechtskonform durchzuführen, sollten Sie folgende Schritte beachten:

1. **Überprüfung der Verfahrensfreiheit**: 
   - Ermitteln Sie, ob Ihre Garage als freistehendes Gebäude der Gebäudeklasse 1 oder 3 zählt. Ist dies der Fall und erfüllt die Garage die Kriterien (Höhe bis zu 7 m und bei Gebäudeklasse 1 nicht mehr als zwei Nutzungseinheiten von insgesamt nicht mehr als 400 m²), könnte der Abriss verfahrensfrei sein.

2. **Denkmalschutz**: 
   - Prüfen Sie, ob die Garage unter Denkmalschutz steht. Informationen dazu finden Sie im Kieler Denkmalkataster.

3. **Anzeige der Beseitigung**: 
   - Falls die Garage nicht unter die verfahrensfreien Anlagen fällt, melden Sie den geplanten Abriss einen Monat vorher bei der Bauaufsichtsbehörde an. Verwenden Sie dafür den Vordruck unter VordruckeLandesbauordnung_Anlage1.pdf auf der Webseite schleswig-holstein.de.

4. **Standsicherheit benachbarter Gebäude**: 
   - Sollte die Garage angebaut sein, ste

In [27]:
from IPython.display import Markdown

display(Markdown(completion.choices[0].message.content))


Um den Abriss Ihrer Garage in Schleswig-Holstein rechtskonform durchzuführen, sollten Sie folgende Schritte beachten:

1. **Überprüfung der Verfahrensfreiheit**: 
   - Ermitteln Sie, ob Ihre Garage als freistehendes Gebäude der Gebäudeklasse 1 oder 3 zählt. Ist dies der Fall und erfüllt die Garage die Kriterien (Höhe bis zu 7 m und bei Gebäudeklasse 1 nicht mehr als zwei Nutzungseinheiten von insgesamt nicht mehr als 400 m²), könnte der Abriss verfahrensfrei sein.

2. **Denkmalschutz**: 
   - Prüfen Sie, ob die Garage unter Denkmalschutz steht. Informationen dazu finden Sie im Kieler Denkmalkataster.

3. **Anzeige der Beseitigung**: 
   - Falls die Garage nicht unter die verfahrensfreien Anlagen fällt, melden Sie den geplanten Abriss einen Monat vorher bei der Bauaufsichtsbehörde an. Verwenden Sie dafür den Vordruck unter VordruckeLandesbauordnung_Anlage1.pdf auf der Webseite schleswig-holstein.de.

4. **Standsicherheit benachbarter Gebäude**: 
   - Sollte die Garage angebaut sein, stellen Sie sicher, dass die Standsicherheit benachbarter Gebäude nicht beeinträchtigt wird. Dies muss bauaufsichtlich geprüft sein.

5. **Kampfmittelbeseitigung**: 
   - Vor Beginn der Erdarbeiten lassen Sie das Grundstück auf mögliche Kampfmittel untersuchen. Kontaktieren Sie hierfür das Innenministerium des Landes Schleswig-Holstein, Landeskriminalamt – Kampfmittelräumdienst.

6. **Abfallentsorgung**: 
   - Informieren Sie sich über die korrekte Abfallentsorgung bei Abbrucharbeiten. Ein entsprechendes Merkblatt finden Sie unter Merkblatt Abfallentsorgung.

7. **Gebühren**: 
   - Beachten Sie mögliche Gebühren, die in der Baugebührenverordnung (BauGebVO) geregelt sind.

8. **Zuständigkeit und Beratung**: 
   - Für weitere Fragen und zur Terminvereinbarung für eine Bauberatung wenden Sie sich an das Amt für Bauordnung, Vermessung und Geoinformation in Kiel.

Indem Sie diese Punkte beachten, stellen Sie sicher, dass der Abriss Ihrer Garage rechtskonform und sicher durchgeführt wird.